<h1 style="background-color:#800080;font-family:newtimeroman;font-size:550%;text-align:center;border-radius: 15px 50px;padding: 5px">Google PlayStore</h1>


<center><img src="https://www.freepnglogos.com/uploads/google-play-png-logo/google-play-services-png-logo-3.png",height='500',width='600'></center>

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Importing modules and Loading datasets</h1>
<a id=1></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
!pip install CurrencyConverter
from currency_converter import CurrencyConverter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  RandomForestRegressor
from keras.layers import Dense,BatchNormalization,Flatten
from keras.models import Sequential
import keras
from keras.metrics import mean_absolute_error
warnings.filterwarnings('ignore')
playstore=pd.read_csv('../input/google-playstore-apps/Google-Playstore.csv',parse_dates=['Last Updated','Released'])
playstore.head()

In [ ]:
playstore.describe()

In [ ]:
playstore.describe(include='O')

In [ ]:
playstore.info()

In [ ]:
playstore.drop(['App Id','Developer Website','Developer Email','Developer Id','Privacy Policy'],axis=1,inplace=True)
playstore.head()

### The least important columns like App Id, Developer Website,etc are deleted.

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Train test spliting of data</h1>
<a id=2></a>

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(playstore,test_size=0.15,random_state=42)

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Dealing with missing values</h1>
<a id=3></a>

In [ ]:
for col in train_data.columns:
    null=train_data[col].isnull().sum()
    print('{0} have {1}   null value'.format(col,null))

In [ ]:
train_data.fillna(method='bfill',inplace=True)

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Categorical type columns</h1>
<a id=4></a>

In [ ]:
categorical_columns=[]
for col in train_data.columns:
    if train_data[col].dtype=='O':
        categorical_columns.append(col)
categorical_columns

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Installs</h1>
<a id=5></a>

In [ ]:
def string_to_numeric(data):
    data=data.replace(',','')
    data=data[:-1]
    return int(data)

train_data['Installs']=train_data['Installs'].apply(string_to_numeric)

In [ ]:
plt.figure(figsize=(20,15))
sns.countplot(train_data['Installs'])
plt.plot()

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Size of App</h1>
<a id=5></a>

In [ ]:
def define_size(data):
    data=data.replace(',','')
    num=data[:-1]
    if data[-1]=='K' or data[-1]=='k' :
        return float(num)
    elif data[-1]=='M' or data[-1]=='m':
        return float(num)*1000
    elif data[-1]=='G' or data[-1]=='g':
        return float(num)*1000000
    else:
        return 1000
        

### Function to convert every size of App to kb.

In [ ]:
train_data['Size']=train_data['Size'].apply(define_size)

In [ ]:
plt.figure(figsize=(20,15))
sns.countplot(train_data['Size'])
plt.plot()

In [ ]:
plt.figure(figsize=(20,15))
sns.lineplot(train_data['Size'],train_data['Installs'])
plt.plot()

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Content Rating</h1>
<a id=6></a>

In [ ]:
train_data['Content Rating'].value_counts()

In [ ]:

plt.pie(train_data['Content Rating'].value_counts(),explode=[0.1,0.2,0.3,0.2,0.5,0.1],autopct='%0.2f%%',
        labels=['Everyone','Teenager','Mature 17+','Everyone 10+','Unrated','Adult only 18+'],startangle=90,radius=3,
       colors=['#00ffff','#800080','#6960ec','#fbb917','#ff00ff','#00ff00'])
plt.title('Content Rating',fontdict={'size':20,'weight':'bold'},loc='right')
plt.plot()

In [ ]:
plt.figure(figsize=(20,15))
sns.countplot(train_data['Content Rating'])
plt.plot()

In [ ]:
lbl_content_rating=LabelEncoder()
train_data['Content Rating']=lbl_content_rating.fit_transform(train_data['Content Rating'])

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Minimum Android Version</h1>
<a id=7></a>

In [ ]:
def min_android(data):
    p=data.split()
    if data=='Varies with device':
        return 4.0
    else:
        return float(p[0][:3])
train_data['Minimum Android']=train_data['Minimum Android'].apply(min_android)

In [ ]:
plt.figure(figsize=(30,15))
sns.countplot(train_data['Minimum Android'])
plt.plot()

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Categories Of App</h1>
<a id=8></a>

In [ ]:
train_data['Category'].value_counts()

In [ ]:
plt.figure(figsize=(30,15))
sns.countplot(train_data['Category'])
plt.plot()

In [ ]:
lbl_category=LabelEncoder()
train_data['Category']=lbl_category.fit_transform(train_data['Category'])

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Currency</h1>
<a id=9></a>

In [ ]:
cc=CurrencyConverter()


def currency_to_INR(data):
    if data not in cc.currencies:
        data=1
    else:
        data=cc.convert(1,data,'INR')
    return data


In [ ]:
train_data['Currency']=train_data['Currency'].apply(currency_to_INR)

In [ ]:
train_data.Price=train_data.Price*train_data.Currency
train_data.Price.value_counts()

### Multipling Price Column with currency to get the actual Price in INR

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Boolean Columns</h1>
<a id=10></a>

In [ ]:
bln=[]
for col in train_data.columns:
    if train_data[col].dtype=='bool':
        bln.append(col)
bln

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Is App is Free?</h1>
<a id=11></a>

In [ ]:
train_data.Free.value_counts()

In [ ]:
plt.pie(train_data.Free.value_counts(),radius=3,autopct='%0.2f%%',explode=[0.2,0.5],colors=['#00ffff','#00ff00'],labels=['Free','Not for Free'],
        startangle=90)
plt.title('Is App is Free?',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
train_data.Free=pd.Categorical(train_data.Free,ordered=True,categories=[False,True]).codes

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Is App is Ad Supported?</h1>
<a id=12></a>

In [ ]:
plt.pie(train_data['Ad Supported'].value_counts(),radius=3,autopct='%0.2f%%',explode=[0.2,0.5],colors=['#800080','#ff00ff'],labels=['Ad Supported','Ad not supported'],
        startangle=90)
plt.title('Is Ad supported?',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
train_data['Ad Supported']=pd.Categorical(train_data['Ad Supported'],ordered=True,categories=[False,True]).codes

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Is App is In Purchases?</h1>
<a id=13></a>

In [ ]:
train_data['In App Purchases'].value_counts()

In [ ]:
plt.pie(train_data['In App Purchases'].value_counts(),radius=3,autopct='%0.2f%%',explode=[0.2,0.5],colors=['#ffa500','#0000a0'],labels=['App not in purchase','App in purchase'],
        startangle=90)
plt.title('Is In App Purchases?',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
train_data['In App Purchases']=pd.Categorical(train_data['In App Purchases'],ordered=True,categories=[False,True]).codes

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Editors Choice</h1>
<a id=14></a>

In [ ]:
train_data['Editors Choice'].value_counts()

In [ ]:
plt.pie(train_data['Editors Choice'].value_counts(),radius=3,autopct='%0.2f%%',explode=[0.2,0.9],colors=['#ffff00','#0000a0'],labels=['Not Editor choice','Editor choice'],
        startangle=90)
plt.title('Is Editors Choice?',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
train_data['Editors Choice']=pd.Categorical(train_data['Editors Choice'],ordered=True,categories=[False,True]).codes

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Released Date</h1>
<a id=15></a>

In [ ]:
released_date_install=pd.concat([train_data['Installs'],train_data['Released']],axis=1)

In [ ]:
plt.figure(figsize=(15,12))
released_date_plot=released_date_install.set_index('Released').resample('3M').mean()
released_date_plot.plot()
plt.title('Released date Vs Installs',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
plt.figure(figsize=(15,10))
released_date_plot.plot(kind='kde')
plt.title('Released date Vs Installs',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
released_date_rating=pd.concat([train_data['Rating'],train_data['Released']],axis=1)

In [ ]:
plt.figure(figsize=(15,12))
released_date_rating_plot=released_date_rating.set_index('Released').resample('3M').mean()
released_date_rating_plot.plot()
plt.title('Released date Vs Rating',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
plt.figure(figsize=(15,10))
released_date_rating_plot.plot(kind='kde')
plt.title('Released date Vs Rating',fontdict={'size':20,'weight':'bold'})
plt.plot()

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Last Updated Date</h1>
<a id=16></a>

In [ ]:
last_update_install=pd.concat([train_data['Installs'],train_data['Last Updated']],axis=1)

In [ ]:
plt.figure(figsize=(15,12))
update_date_install_plot=last_update_install.set_index('Last Updated').resample('3M').mean()
update_date_install_plot.plot()
plt.title('Last Updated date Vs Rating',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
plt.figure(figsize=(15,10))
update_date_install_plot.plot(kind='kde')
plt.title('Last Updated date Vs Rating',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
updated_date_rating=pd.concat([train_data['Rating'],train_data['Last Updated']],axis=1)

In [ ]:
plt.figure(figsize=(15,12))
updated_date_rating_plot=updated_date_rating.set_index('Last Updated').resample('3M').mean()
updated_date_rating_plot.plot()
plt.title('Last Updated Vs Rating',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
plt.figure(figsize=(15,10))
updated_date_rating_plot.plot(kind='kde')
plt.title('Last Updated date Vs Rating',fontdict={'size':20,'weight':'bold'})
plt.plot()

In [ ]:
train_data.drop(['Released','Last Updated','Currency'],inplace=True,axis=1)

In [ ]:
train_data.head()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(train_data.corr(),annot=True)
plt.plot()

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Cleaning Test Data </h1>
<a id=17></a>

In [ ]:
test_data.fillna(method='bfill',inplace=True)

In [ ]:
test_data['Installs']=test_data['Installs'].apply(string_to_numeric)
test_data['Category']=lbl_category.transform(test_data['Category'])
test_data['Content Rating']=lbl_content_rating.transform(test_data['Content Rating'])
test_data['Minimum Android']=test_data['Minimum Android'].apply(min_android)
test_data['Size']=test_data['Size'].apply(define_size)
test_data['Currency']=test_data['Currency'].apply(currency_to_INR)
test_data['Editors Choice']=pd.Categorical(test_data['Editors Choice'],ordered=True,categories=[False,True]).codes
test_data['In App Purchases']=pd.Categorical(test_data['In App Purchases'],ordered=True,categories=[False,True]).codes
test_data['Ad Supported']=pd.Categorical(test_data['Ad Supported'],ordered=True,categories=[False,True]).codes
test_data['Free']=pd.Categorical(test_data['Free'],ordered=True,categories=[False,True]).codes
test_data['Price']=test_data['Price']*test_data['Currency']
test_data.drop(['Released','Last Updated','Currency'],inplace=True,axis=1)

In [ ]:
test_data.info()

In [ ]:
y_test=test_data.iloc[:,2]
x_test=test_data.drop(['Rating','Minimum Installs','Maximum Installs','App Name'],axis=1)


y_train=train_data.iloc[:,2]
x_train=train_data.drop(['Rating','Minimum Installs','Maximum Installs','App Name'],axis=1)

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Model Selection </h1>
<a id=18></a>

<h1 style="background-color:#00ff00;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Linear Regression </h1>
<a id=19></a>

In [ ]:

model_1=LinearRegression()
model_1.fit(x_train,y_train)
print('r2_score of training data',r2_score(model_1.predict(x_train).round(1),y_train))
print('r2_score of testing data',r2_score(model_1.predict(x_test).round(1),y_test))

In [ ]:

print('mean absolute error of training data',mean_absolute_error(model_1.predict(x_train).round(1),y_train))
print('mean absolute error of testing data',mean_absolute_error(model_1.predict(x_test).round(1),y_test))

<h1 style="background-color:#00ff00;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Decision Tree Regression </h1>
<a id=20></a>

In [ ]:
model_2=DecisionTreeRegressor(max_depth=9)
model_2.fit(x_train,y_train)
print('r2_score of training data',r2_score(model_2.predict(x_train).round(1),y_train))
print('r2_score of testing data',r2_score(model_2.predict(x_test).round(1),y_test))

In [ ]:

print('mean absolute error of training data',mean_absolute_error(model_2.predict(x_train).round(1),y_train))
print('mean absolute error of testing data',mean_absolute_error(model_2.predict(x_test).round(1),y_test))

<h1 style="background-color:#00ff00;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Random Forest Regression </h1>
<a id=21></a>

In [ ]:
model_3=RandomForestRegressor(max_depth=9)
model_3.fit(x_train,y_train)
print('r2_score of training data',r2_score(model_3.predict(x_train).round(1),y_train))
print('r2_score of testing data',r2_score(model_3.predict(x_test).round(1),y_test))

In [ ]:

print('mean absolute error of training data',mean_absolute_error(model_3.predict(x_train).round(1),y_train))
print('mean absolute error of testing data',mean_absolute_error(model_3.predict(x_test).round(1),y_test))

<h1 style="background-color:#00ff00;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">XGBRegression </h1>
<a id=21></a>

In [ ]:
from xgboost import XGBRegressor
model_4=XGBRegressor()
model_4.fit(x_train,y_train)
print('r2_score of training data',r2_score(model_4.predict(x_train).round(1),y_train))
print('r2_score of testing data',r2_score(model_4.predict(x_test).round(1),y_test))

In [ ]:

print('mean absolute error of training data',mean_absolute_error(model_4.predict(x_train).round(1),y_train))
print('mean absolute error of testing data',mean_absolute_error(model_4.predict(x_test).round(1),y_test))

<h1 style="background-color:#00ff00;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">ANN </h1>
<a id=22></a>

In [ ]:
model_5=Sequential()
model_5.add(Dense(1,input_shape=x_train.shape))

In [ ]:

model_5.compile(optimizer=keras.optimizers.RMSprop(.001),loss='mse',metrics=[keras.metrics.mean_absolute_error])
r=model_5.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5,batch_size=100)

In [ ]:
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])

In [ ]:

print('r2_score of training data',r2_score(model_5.predict(x_train),y_train))
print('r2_score of testing data',r2_score(model_5.predict(x_test),y_test))

<h1 style="background-color:#00ffff;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Conclusion</h1>
<a id=22></a>

### Here it is found that XG Boost and Random Forest are some of the better techniques that could be used to predict output.

<h1 style="background-color:#f660ab;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 10px 10px;padding: 3px">If you liked this notebook or it helped you, please upvote!!</h1>